# 第四章作业

### 1.使用tensor初始化一个1x3的矩阵M和2x1的矩阵N，然后对两个矩阵做减法（用三种不同的方式，提示：直接相减、torch.nn、inplace原地操作），最后分析三种方式的不同。

定义两个矩阵mat1,mat2·

In [1]:
import torch
import torch.nn as nn

mat1 = torch.rand(1, 3)
mat2 = torch.rand(2, 1)
print(f'mat1:{mat1}, mat2:{mat2}')

mat1:tensor([[0.1268, 0.3749, 0.8659]]), mat2:tensor([[0.7299],
        [0.3648]])


进行直接相减操作

In [2]:
print('----------------直接相减----------------')
print(f'mat1-mat2={mat1 - mat2}')

----------------直接相减----------------
mat1-mat2=tensor([[-0.6031, -0.3551,  0.1360],
        [-0.2380,  0.0101,  0.5011]])


进行原地操作inplace

In [3]:
print('----------------原地操作inplace----------------')
# 调整张量形状以使它们兼容
# 扩展mat1和mat2，使它们可以广播到相同形状
mat1_expanded = mat1.expand(2, 3)
mat2_expanded = mat2.expand(2, 3)
# 尝试进行原地操作，但由于原地操作不能改变形状，使用扩展后的张量
mat1_inplace_adjusted = mat1_expanded.clone()
mat1_inplace_adjusted.sub_(mat2_expanded)
print(f'mat1-mat2={mat1_inplace_adjusted}')

----------------原地操作inplace----------------
mat1-mat2=tensor([[-0.6031, -0.3551,  0.1360],
        [-0.2380,  0.0101,  0.5011]])


进行torch.nn操作

In [4]:
print('----------------使用torch.nn----------------')
# 创建一个线性层，权重为单位矩阵，偏置设置为mat2[0]
linear = nn.Linear(3, 3, bias=True)
linear.weight.data = torch.eye(3)
linear.bias.data = -mat2[0].expand(1, 3).contiguous()
# 创建一个线性层，权重为单位矩阵，偏置设置为mat2[1]
linear2 = nn.Linear(3, 3, bias=True)
linear2.weight.data = torch.eye(3)
linear2.bias.data = -mat2[1].expand(1, 3).contiguous()
# 应用线性层
mat1_expanded = mat1.expand(2, 3)  # 扩展 mat1 以匹配 mat2 的形状
result_1 = linear(mat1_expanded[0].view(1, -1))
result_2 = linear2(mat1_expanded[1].view(1, -1))
result = torch.cat((result_1, result_2), dim=0)
print(f'mat1 - mat2 = {result}')

----------------使用torch.nn----------------
mat1 - mat2 = tensor([[-0.6031, -0.3551,  0.1360],
        [-0.2380,  0.0101,  0.5011]], grad_fn=<CatBackward0>)


### 4.请实现两个数的加法，即计算 A+B并输出，其中 A 是常量，B是占位符，数据类型自定。

In [14]:
import torch

# 定义常量 A
A = torch.tensor(5)

# 定义占位符 B
B = torch.tensor(0)  # 初始化为 0，稍后赋值

print(A)
print(B)

tensor(5)
tensor(0)


In [15]:
# 定义一个函数来执行加法操作
def add_tensors(tensor_A, tensor_B):
    return tensor_A + tensor_B

# 假设我们现在有一个值为 3 的 B
B = torch.tensor(3)

# 执行加法操作并打印结果
result = add_tensors(A, B)
print("结果:", result.item())

结果: 8


### 5.请实现一个矩阵乘法，数据类型和规模自定，并分别使用 CPU和GPU执行

In [8]:
import torch
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 定义两个较大的矩阵以突出性能差异
matrix1 = torch.randn(1000, 1000)
matrix2 = torch.randn(1000, 1000)

# CPU 上的矩阵乘法
start_time_cpu = time.time()
result_cpu = torch.matmul(matrix1, matrix2)
end_time_cpu = time.time()
cpu_time = end_time_cpu - start_time_cpu

# 检查是否有可用的 GPU
if torch.cuda.is_available():
    # 将矩阵移到 GPU
    matrix1_gpu = matrix1.to('cuda')
    matrix2_gpu = matrix2.to('cuda')

    # GPU 上的矩阵乘法
    torch.cuda.synchronize()  # 确保所有 CUDA 核心同步
    start_time_gpu = time.time()
    result_gpu = torch.matmul(matrix1_gpu, matrix2_gpu)
    torch.cuda.synchronize()  # 等待 GPU 计算完成
    end_time_gpu = time.time()
    gpu_time = end_time_gpu - start_time_gpu
else:
    gpu_time = "GPU 不可用"

print(f'cpu_time is : {cpu_time}')
print(f'gpu_time is : {gpu_time}')

cpu_time is : 0.01898336410522461
gpu_time is : 0.006000518798828125


### 6. 请 调 研 了 解 常 用 的 图 像 数 据 预 处 理 和 数 据 增 强 方 法 。 实现一个函数 ， 从ImageNet2012_val数据集中选择一张图片文件并读入数据，调整为 (256，256，3)大小的图片，然后居中裁剪为(224，224，3) 大小的图片;再实现一个函数，读入数据后居中裁剪为(0.875* width，0.875 * height，3) 大小的图片，再调整为 (224，224，3) 大小的图片。

In [2]:
from PIL import Image

def process_image_1(image_path):
    """ 调整为 (256, 256, 3) 然后裁剪为 (224, 224, 3) """
    with Image.open(image_path) as img:
        # 调整大小为 (256, 256)
        img = img.resize((256, 256))
        # 居中裁剪为 (224, 224)
        left = (256 - 224) / 2
        top = (256 - 224) / 2
        right = (256 + 224) / 2
        bottom = (256 + 224) / 2
        img_cropped = img.crop((left, top, right, bottom))
        return img_cropped

def process_image_2(image_path):
    """ 居中裁剪为 (0.875 * width, 0.875 * height) 然后调整为 (224, 224, 3) """
    with Image.open(image_path) as img:
        width, height = img.size
        new_width = int(width * 0.875)
        new_height = int(height * 0.875)
        # 居中裁剪
        left = (width - new_width) / 2
        top = (height - new_height) / 2
        right = (width + new_width) / 2
        bottom = (height + new_height) / 2
        img_cropped = img.crop((left, top, right, bottom))
        # 调整大小为 (224, 224)
        img_resized = img_cropped.resize((224, 224))
        return img_resized

# 从本地文件 "sample.jpg" 中读取并处理图片
image_path = "./sample.jpg"

# 处理图片
processed_image_1 = process_image_1(image_path)
processed_image_2 = process_image_2(image_path)

# 显示处理后的图片
processed_image_1.show()
processed_image_1.save('out1.jpg')
processed_image_2.show()
processed_image_2.save('out2.jpg')

(224, 224)
